In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.3 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 

In [ ]:
import pandas as pd

df = pd.DataFrame({'Salarié': ['Eric', 'Elise', 'Hapsa', 'Xin'], 'Genre': ['Homme', 'Femme', 'Femme', 'Homme'], 'groupe': ['Technicien',  'Ingénieur','PhD', 
     'Technicien'],'date_embauche': [2016, 1988, 2012, 2001], 'salaire': [30000, 100000, 60000, 60000]})
df

,Salarié,Genre,groupe,date_embauche,salaire
0,Eric,Homme,Technicien,2016,30000
1,Elise,Femme,Ingénieur,1988,100000
2,Hapsa,Femme,PhD,2012,60000
3,Xin,Homme,Technicien,2001,60000


In [ ]:
df1 = spark.createDataFrame(df)
df1.show()

+-------+-----+----------+-------------+-------+
|Salarié|Genre|    groupe|date_embauche|salaire|
+-------+-----+----------+-------------+-------+
|   Eric|Homme|Technicien|         2016|  30000|
|  Elise|Femme| Ingénieur|         1988| 100000|
|  Hapsa|Femme|       PhD|         2012|  60000|
|    Xin|Homme|Technicien|         2001|  60000|
+-------+-----+----------+-------------+-------+



In [ ]:
from pyspark.ml.feature import StringIndexer
Genre_index = StringIndexer(inputCol="Genre", outputCol="Genre_num").fit(df1)
df1 = Genre_index.transform(df1)
df1.show()

+-------+-----+----------+-------------+-------+---------+
|Salarié|Genre|    groupe|date_embauche|salaire|Genre_num|
+-------+-----+----------+-------------+-------+---------+
|   Eric|Homme|Technicien|         2016|  30000|      1.0|
|  Elise|Femme| Ingénieur|         1988| 100000|      0.0|
|  Hapsa|Femme|       PhD|         2012|  60000|      0.0|
|    Xin|Homme|Technicien|         2001|  60000|      1.0|
+-------+-----+----------+-------------+-------+---------+



In [ ]:
from pyspark.ml.feature import StringIndexer
groupe_index = StringIndexer(inputCol="groupe", outputCol="groupe_num").fit(df1)
df1 = groupe_index.transform(df1)

from pyspark.ml.feature import OneHotEncoder
groupe_encod = OneHotEncoder(inputCol="groupe_num", outputCol="groupe_vecteur")
df1 = groupe_encod.transform(df1)
df1.show()

+-------+-----+----------+-------------+-------+---------+-------------+----------+--------------+
|Salarié|Genre|    groupe|date_embauche|salaire|Genre_num|Genre_vecteur|groupe_num|groupe_vecteur|
+-------+-----+----------+-------------+-------+---------+-------------+----------+--------------+
|   Eric|Homme|Technicien|         2016|  30000|      1.0|    (1,[],[])|       0.0| (2,[0],[1.0])|
|  Elise|Femme| Ingénieur|         1988| 100000|      0.0|(1,[0],[1.0])|       2.0|     (2,[],[])|
|  Hapsa|Femme|       PhD|         2012|  60000|      0.0|(1,[0],[1.0])|       1.0| (2,[1],[1.0])|
|    Xin|Homme|Technicien|         2001|  60000|      1.0|    (1,[],[])|       0.0| (2,[0],[1.0])|
+-------+-----+----------+-------------+-------+---------+-------------+----------+--------------+

